In [1]:
from importlib.util import find_spec
%load_ext autoreload
%autoreload 2

%matplotlib inline

if find_spec("qml_hep_lhc") is None:
    import sys
    sys.path.append('../..')

In [2]:
from qml_hep_lhc.data import ElectronPhoton, MNIST, QuarkGluon
from qml_hep_lhc.models import ResnetQ50
from tensorflow.keras.callbacks import ReduceLROnPlateau
import argparse
import wandb
import numpy as np

import matplotlib.pyplot as plt
import time

2022-09-25 15:06:33.461469: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-25 15:06:33.461536: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-09-25 15:06:40.936379: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-09-25 15:06:40.936467: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (bhagvada): /proc/driver/nvidia/version does not exist
2022-09-25 15:06:40.936922: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
T

In [24]:
args = argparse.Namespace()

# Data
args.standardize = 1
args.resize = [32,32]
args.binary_data = [0,1]
args.batch_size = 1024
args.validation_split = 0.05
args.hinge_labels = 1
args.use_quantum = 1
args.to_rgb = 1

# Base Model
args.learning_rate = 1e-3
args.epochs = 20

# Quantum CNN Parameters
args.n_layers = 1
args.feature_map = 'AmplitudeMap'
args.ansatz_class = 'Farhi'

In [20]:
data = MNIST(args)
data.prepare_data()
data.setup()
print(data)

Binarizing data...
Binarizing data...
Converting to RGB...
Resizing data...
Resizing data...
Standardizing data...
Hinge labels...
Hinge labels...

Dataset :MNIST
╒════════╤════════════════════╤══════════════════╤═══════════════════╤═════════════╕
│ Data   │ Train size         │ Val size         │ Test size         │ Dims        │
╞════════╪════════════════════╪══════════════════╪═══════════════════╪═════════════╡
│ X      │ (12031, 32, 32, 3) │ (634, 32, 32, 3) │ (2115, 32, 32, 3) │ (32, 32, 3) │
├────────┼────────────────────┼──────────────────┼───────────────────┼─────────────┤
│ y      │ (12031,)           │ (634,)           │ (2115,)           │ (1,)        │
╘════════╧════════════════════╧══════════════════╧═══════════════════╧═════════════╛

╒══════════════╤═══════╤═════════╤════════╤═══════╤══════════════════════════╕
│ Type         │   Min │     Max │   Mean │   Std │ Samples for each class   │
╞══════════════╪═══════╪═════════╪════════╪═══════╪══════════════════════════╡
│ Tr

In [15]:
data.config()

{'input_dims': (32, 32, 3), 'output_dims': (1,), 'mapping': [0, 1]}

In [25]:
model = ResnetQ50(data.config(), args)

../../qml_hep_lhc/encodings/amplitude.py:13: UserWarning: AmplitudeMap currently does not normalize the input unless padding is needed.
User must manually normalize the input.
  "AmplitudeMap currently does not normalize the input unless padding is needed.\nUser must manually normalize the input."


In [26]:
model.compile()
model.build_graph().summary()

Model: "ResnetQ50"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 resnet50 (Functional)       (None, 1, 1, 2048)        23587712  
                                                                 
 flatten_6 (Flatten)         (None, 2048)              0         
                                                                 
 dropout_2 (Dropout)         (None, 2048)              0         
                                                                 
 dense_4 (Dense)             (None, 512)               1049088   
                                                                 
 dense_5 (Dense)             (None, 16)                8208      
                                                                 
 TwoLayerPQC (TwoLayerPQC)   (None, 1)                 27

In [27]:
lr_scheduler_callback = ReduceLROnPlateau(monitor='val_loss',
                                              factor=np.sqrt(0.1),
                                              patience=5,
                                              min_delta=0.0001,
                                              min_lr=1e-8)
callbacks = [lr_scheduler_callback]
model.fit(data, callbacks)

Epoch 1/20


KeyboardInterrupt: 

In [ ]:
model.test(data, callbacks=callbacks)